# RePath Model Release Workflow

This notebook walks through building and validating a semver model release bundle, then publishing a GitHub release.

## 1. Prerequisites
- Run this notebook from the `repath-model` repo root.
- `gh` CLI authenticated (`gh auth status`).
- `release-notes.md` updated for the target release.

In [ ]:
import os
import subprocess
from datetime import datetime
from pathlib import Path


def resolve_root(env_key: str, candidates):
    value = os.environ.get(env_key, "").strip()
    if value:
        return Path(value).expanduser().resolve()
    for candidate in candidates:
        candidate = candidate.resolve()
        if candidate.exists():
            return candidate
    return candidates[0].resolve()


ROOT = Path.cwd()
VERSION = "1.0.0"  # set target version
TAG = f"v{VERSION}"
SOURCE_RUN_ID = os.environ.get("SOURCE_RUN_ID", datetime.now().strftime("%Y%m%d-%H%M%S"))

MODEL_ASSETS_ROOT = resolve_root(
    "REPATH_MODEL_ASSETS_ROOT",
    [ROOT / "assets/models", ROOT / "../repath-mobile/assets/models"],
)
MODEL_PATH = MODEL_ASSETS_ROOT / "yolo-repath.tflite"
LABELS_PATH = MODEL_ASSETS_ROOT / "yolo-repath.labels.json"
NOTES_PATH = Path("release-notes.md").resolve()

print({
    "root": str(ROOT),
    "version": VERSION,
    "source_run_id": SOURCE_RUN_ID,
    "model_assets_root": str(MODEL_ASSETS_ROOT),
    "model": str(MODEL_PATH),
    "labels": str(LABELS_PATH),
    "release_notes": str(NOTES_PATH),
})


In [ ]:
# Build release bundle
subprocess.run([
    "python3", "scripts/release/build_release.py",
    "--version", VERSION,
    "--model", str(MODEL_PATH),
    "--labels", str(LABELS_PATH),
    "--source-run-id", SOURCE_RUN_ID,
    "--notes-file", str(NOTES_PATH),
], check=True)

In [ ]:
# Verify generated release manifest checksums
manifest = ROOT / f"dist/releases/{TAG}/release-manifest-{TAG}.json"
subprocess.run([
    "python3", "scripts/release/verify_release.py",
    "--manifest", str(manifest),
], check=True)

In [ ]:
# Inspect bundle files
release_dir = ROOT / f"dist/releases/{TAG}"
for p in sorted(release_dir.glob("*")):
    print(p.name, p.stat().st_size)

## 2. Publish to GitHub Releases
Uncomment and run these cells when ready to publish.

In [ ]:
# subprocess.run(["git", "tag", TAG], check=True)
# subprocess.run(["git", "push", "origin", TAG], check=True)

In [ ]:
# subprocess.run([
#     "gh", "release", "create", TAG,
#     f"dist/releases/{TAG}/yolo-repath-{TAG}.tflite",
#     f"dist/releases/{TAG}/yolo-repath-{TAG}.labels.json",
#     f"dist/releases/{TAG}/release-manifest-{TAG}.json",
#     f"dist/releases/{TAG}/SHA256SUMS",
#     "--repo", "krispeterson/repath-model",
#     "--title", f"RePath Model {TAG}",
#     "--notes-file", str(NOTES_PATH),
# ], check=True)